In [1]:
import dicttoxml
from xml.dom.minidom import parseString
import xml.etree.ElementTree as ET

In [2]:
from xml.dom import minidom
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [3]:
p0="100000.0"

In [4]:
f = open('thermo.txt','r')
lines = f.readlines()
f.close()

In [5]:
content = []
for line in lines:
    l = line.strip().split()
    content.append(l)
content

[['THERMO'],
 ['300.000', '1000.000', '5000.000'],
 ['!', 'GRI-Mech', 'Version', '3.0', 'Thermodynamics', 'released', '7/30/99'],
 ['!', 'NASA', 'Polynomial', 'format', 'for', 'CHEMKIN-II'],
 ['!', 'see', 'README', 'file', 'for', 'disclaimer'],
 ['O', 'L', '1/90O', '1', 'G', '200.000', '3500.000', '1000.000', '1'],
 ['2.56942078E+00-8.59741137E-05',
  '4.19484589E-08-1.00177799E-11',
  '1.22833691E-15',
  '2'],
 ['2.92175791E+04',
  '4.78433864E+00',
  '3.16826710E+00-3.27931884E-03',
  '6.64306396E-06',
  '3'],
 ['-6.12806624E-09',
  '2.11265971E-12',
  '2.91222592E+04',
  '2.05193346E+00',
  '4'],
 ['O2', 'TPIS89O', '2', 'G', '200.000', '3500.000', '1000.000', '1'],
 ['3.28253784E+00',
  '1.48308754E-03-7.57966669E-07',
  '2.09470555E-10-2.16717794E-14',
  '2'],
 ['-1.08845772E+03',
  '5.45323129E+00',
  '3.78245636E+00-2.99673416E-03',
  '9.84730201E-06',
  '3'],
 ['-9.68129509E-09', '3.24372837E-12-1.06394356E+03', '3.65767573E+00', '4'],
 ['H', 'L', '7/88H', '1', 'G', '200.000', '

In [6]:
species = ['O','O2','H','H2','OH','H2O','HO2','H2O2']

In [7]:
species_i = []
first = [c[0] for c in content if c]
for i, f in enumerate(first):
    if f == 'THERMO':
        thermo_i = i+1
    for j, s in enumerate(species):
        if f == s:
            species_i.append(i)
            del species[j]
species_i

[5, 9, 13, 17, 21, 25, 29, 33]

In [8]:
temperature = content[thermo_i]
temperature

['300.000', '1000.000', '5000.000']

In [9]:
coeffs = []
for i in species_i:
    numbers = content[i+1][:-1] + content[i+2][:-1] + content[i+3][:-1]
    coeffs.append(numbers)
coeffs

[['2.56942078E+00-8.59741137E-05',
  '4.19484589E-08-1.00177799E-11',
  '1.22833691E-15',
  '2.92175791E+04',
  '4.78433864E+00',
  '3.16826710E+00-3.27931884E-03',
  '6.64306396E-06',
  '-6.12806624E-09',
  '2.11265971E-12',
  '2.91222592E+04',
  '2.05193346E+00'],
 ['3.28253784E+00',
  '1.48308754E-03-7.57966669E-07',
  '2.09470555E-10-2.16717794E-14',
  '-1.08845772E+03',
  '5.45323129E+00',
  '3.78245636E+00-2.99673416E-03',
  '9.84730201E-06',
  '-9.68129509E-09',
  '3.24372837E-12-1.06394356E+03',
  '3.65767573E+00'],
 ['2.50000001E+00-2.30842973E-11',
  '1.61561948E-14-4.73515235E-18',
  '4.98197357E-22',
  '2.54736599E+04-4.46682914E-01',
  '2.50000000E+00',
  '7.05332819E-13-1.99591964E-15',
  '2.30081632E-18-9.27732332E-22',
  '2.54736599E+04-4.46682853E-01'],
 ['3.33727920E+00-4.94024731E-05',
  '4.99456778E-07-1.79566394E-10',
  '2.00255376E-14',
  '-9.50158922E+02-3.20502331E+00',
  '2.34433112E+00',
  '7.98052075E-03-1.94781510E-05',
  '2.01572094E-08-7.37611761E-12-9.179

In [10]:
total_list = []
for lst in coeffs:
    new_list = []
    for i, n in enumerate(lst):
        if len(n) == 29:
            lst[i] = n[:14] + ' ' + n[14:]
        elif len(n) == 30:
            lst[i] = n[:15] + ' ' + n[15:]
        elif len(n) == 44:
            lst[i] = n[:14] + ' ' + n[14:29] + ' ' + n[29:]
        new = lst[i].strip().split()
        new_list.append(new)
    concat_list = [j for i in new_list for j in i]
    total_list.append(concat_list)

total_list

[['2.56942078E+00',
  '-8.59741137E-05',
  '4.19484589E-08',
  '-1.00177799E-11',
  '1.22833691E-15',
  '2.92175791E+04',
  '4.78433864E+00',
  '3.16826710E+00',
  '-3.27931884E-03',
  '6.64306396E-06',
  '-6.12806624E-09',
  '2.11265971E-12',
  '2.91222592E+04',
  '2.05193346E+00'],
 ['3.28253784E+00',
  '1.48308754E-03',
  '-7.57966669E-07',
  '2.09470555E-10',
  '-2.16717794E-14',
  '-1.08845772E+03',
  '5.45323129E+00',
  '3.78245636E+00',
  '-2.99673416E-03',
  '9.84730201E-06',
  '-9.68129509E-09',
  '3.24372837E-12',
  '-1.06394356E+03',
  '3.65767573E+00'],
 ['2.50000001E+00',
  '-2.30842973E-11',
  '1.61561948E-14',
  '-4.73515235E-18',
  '4.98197357E-22',
  '2.54736599E+04',
  '-4.46682914E-01',
  '2.50000000E+00',
  '7.05332819E-13',
  '-1.99591964E-15',
  '2.30081632E-18',
  '-9.27732332E-22',
  '2.54736599E+04',
  '-4.46682853E-01'],
 ['3.33727920E+00',
  '-4.94024731E-05',
  '4.99456778E-07',
  '-1.79566394E-10',
  '2.00255376E-14',
  '-9.50158922E+02',
  '-3.20502331E+00

In [18]:
species = ['O','O2','H','H2','OH','H2O','HO2','H2O2']
d1 = {}
for i, t in enumerate(total_list):
    d1[species[i]] = {}
    d1[species[i]]['high'] = t[:7]
    d1[species[i]]['low'] = t[7:]
d1

{'H': {'high': ['2.50000001E+00',
   '-2.30842973E-11',
   '1.61561948E-14',
   '-4.73515235E-18',
   '4.98197357E-22',
   '2.54736599E+04',
   '-4.46682914E-01'],
  'low': ['2.50000000E+00',
   '7.05332819E-13',
   '-1.99591964E-15',
   '2.30081632E-18',
   '-9.27732332E-22',
   '2.54736599E+04',
   '-4.46682853E-01']},
 'H2': {'high': ['3.33727920E+00',
   '-4.94024731E-05',
   '4.99456778E-07',
   '-1.79566394E-10',
   '2.00255376E-14',
   '-9.50158922E+02',
   '-3.20502331E+00'],
  'low': ['2.34433112E+00',
   '7.98052075E-03',
   '-1.94781510E-05',
   '2.01572094E-08',
   '-7.37611761E-12',
   '-9.17935173E+02',
   '6.83010238E-01']},
 'H2O': {'high': ['3.03399249E+00',
   '2.17691804E-03',
   '-1.64072518E-07',
   '-9.70419870E-11',
   '1.68200992E-14',
   '-3.00042971E+04',
   '4.96677010E+00'],
  'low': ['4.19864056E+00',
   '-2.03643410E-03',
   '6.52040211E-06',
   '-5.48797062E-09',
   '1.77197817E-12',
   '-3.02937267E+04',
   '-8.49032208E-01']},
 'H2O2': {'high': ['4.1650

In [12]:
# xml = dicttoxml.dicttoxml(d1, custom_root='ctml', attr_type=False)

In [13]:
# dom = parseString(xml)
# print(dom.toprettyxml())

In [14]:
root = ET.Element("ctml")
doc = ET.SubElement(root, "phase", id="gri30")
ET.SubElement(doc, "speciesArray", datasrc="#species_data").text = 'O O2 H H2 OH H2O HO2 H2O2'

In [15]:
doc2 = ET.SubElement(root, "speciesData", id="species_data")
# H = ET.SubElement(doc2, "species", name="H")
# H_thermo = ET.SubElement(H, "thermo")
# H_NASA_high = ET.SubElement(H_thermo, "NASA", TMAX=temperature[1], TMIN=temperature[0], P0=p0)
# H_fa_high = ET.SubElement(H_NASA_high, "floatArray", name="coeffs", size="7").text = ','.join(d1['H']['high'])
# H_NASA_low = ET.SubElement(H_thermo, "NASA", TMAX=temperature[2], TMIN=temperature[1], P0=p0)
# H_fa_low = ET.SubElement(H_NASA_low, "floatArray", name="coeffs", size="7").text = ','.join(d1['H']['low'])

In [16]:
for specie in species:
    s = ET.SubElement(doc2, "species", name=specie)
    s_thermo = ET.SubElement(s, "thermo")
    s_NASA_high = ET.SubElement(s_thermo, "NASA", TMAX=temperature[1], TMIN=temperature[0], P0=p0)
    s_fa_high = ET.SubElement(s_NASA_high, "floatArray", name="coeffs", size="7").text = ','.join(d1[specie]['high'])
    s_NASA_low = ET.SubElement(s_thermo, "NASA", TMAX=temperature[2], TMIN=temperature[1], P0=p0)
    s_fa_low = ET.SubElement(s_NASA_low, "floatArray", name="coeffs", size="7").text = ','.join(d1[specie]['low'])

In [17]:
tree = ET.ElementTree(root)
tree.write("thermo.xml")
with open("thermo.xml", 'w') as f:
    f.write(prettify(root))